In [ ]:
! pip install torchaudio
! pip install huggingface_hub
! pip install datasets[audio]
! pip install ffmpeg
! pip install pydub

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
import os

from pydub import AudioSegment
import pandas as pd
from google.colab import drive
from datasets import Audio, Dataset, load_dataset

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
hugginface_repo = "lpetrl/UKR-Medical-Speech"
to_folder_data = "/content/gdrive/MyDrive/DS/data/DS/dataset/resampling"

In [ ]:
cd /content/gdrive/MyDrive/DS/data/Structured Audio Segments

/content/gdrive/.shortcut-targets-by-id/1EQAQJTis_-sZ5DYs8rPRCU8NbWDSVUK8/Structured_Audio_Segments


In [ ]:
users_df = pd.read_excel("anotations.xlsx")
users_df = users_df.dropna()
users_df

,index,user_id,description
0,7_1,474721978.0,Хвора К. 22 роки поступила у лікарню
1,7_2,474721978.0,зі скаргами на напади ядухи до трьох-чотирьох ...
2,7_3,474721978.0,"кашель з виділенням густого прозорого харкотиння,"
3,7_4,474721978.0,"задишку, при огляді ціаноз губ,"
4,7_5,474721978.0,"грудна клітка бочкоподібної форми, набряклі шийні"
...,...,...,...
5689,1028.4,575239699.0,у 480 дітей виявлені тимчасові медичні протипо...
5690,1028.5,575239699.0,Яка величина показника “Повнота охоплення проф...
5691,1028.6,575239699.0,"дітей та підлітків, що підлягають щепленням” у..."
5692,1029.1,575239699.0,Серед усієї суми зареєстрованих захворювань мі...


In [ ]:
users_df["user_id"] = users_df["user_id"].astype(int).astype(str)

In [ ]:
import os

folder_path = "/content/gdrive/MyDrive/DS/data/Structured Audio Segments"

file_lists = {}
res = []
for root, dirs, files in os.walk(folder_path):
    wav_files = [file.split(".wav")[0] for file in files if file.lower().endswith(".wav")]

    file_lists[root] = wav_files
    res.extend(wav_files)

len(res)

5663

In [ ]:
users_df = users_df[~users_df['index'].isin(list(set(users_df["index"].values).difference(set(res))))]
users_df.drop_duplicates(subset=["index"], inplace=False)

In [ ]:
import os
root_folder = '/content/gdrive/.shortcut-targets-by-id/14LcMKlutKERAdab7Uz6D5PM1hVqEiILF/dataset/resampling'

df = pd.DataFrame(columns=['index', 'user_id'])

# Функция для обхода файловой системы
def process_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.wav'):
                # Добавляем данные о файле в DataFrame
                df.loc[len(df)] = [file, os.path.basename(root)]

process_folder(root_folder)
df

           index    user_id
0        0_0.wav  474721978
1        1_2.wav  474721978
2        0_2.wav  474721978
3        1_1.wav  474721978
4        2_2.wav  474721978
...          ...        ...
8924  2163_2.wav  572256777
8925  2163_3.wav  572256777
8926  2165_5.wav  572256777
8927  2166_1.wav  572256777
8928  2161_6.wav  572256777

[8929 rows x 2 columns]


In [ ]:
users_df = users_df[users_df['index'].isin(df['index'].str.replace('.wav', ''))]

In [ ]:
# users_df = users_df[~users_df['description'].str.contains(r'\d')]
# users_df

In [ ]:
users_df.rename(columns={"description": "text"}, inplace=True)

<ipython-input-123-1038ecb29bb9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_df.rename(columns={"description": "text"}, inplace=True)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
users_df

,index,user_id,text
0,7_1,474721978,Хвора К. 22 роки поступила у лікарню
1,7_2,474721978,зі скаргами на напади ядухи до трьох-чотирьох ...
2,7_3,474721978,"кашель з виділенням густого прозорого харкотиння,"
3,7_4,474721978,"задишку, при огляді ціаноз губ,"
4,7_5,474721978,"грудна клітка бочкоподібної форми, набряклі шийні"
...,...,...,...
5689,1028.4,575239699,у 480 дітей виявлені тимчасові медичні протипо...
5690,1028.5,575239699,Яка величина показника “Повнота охоплення проф...
5691,1028.6,575239699,"дітей та підлітків, що підлягають щепленням” у..."
5692,1029.1,575239699,Серед усієї суми зареєстрованих захворювань мі...


In [ ]:
train, test = train_test_split(users_df, test_size=0.2, stratify=users_df['user_id'])

In [ ]:
train["audio"] = train.apply(lambda row: f"{row['user_id']}/{row['index']}.wav", axis=1)
test["audio"] = test.apply(lambda row: f"{row['user_id']}/{row['index']}.wav", axis=1)

train.head(2)

,index,user_id,text,audio
3149,632.4,512486219,під час дихання та кашлю об'єктивно дихання,512486219/632.4.wav
3918,726.1,345783892,хворий 24 роки звернувся в поліклініку зі скар...,345783892/726.1.wav


In [ ]:
from datasets import DatasetDict


train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

train_dataset = train_dataset.cast_column("audio", Audio())
test_dataset = test_dataset.cast_column("audio", Audio())

dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

In [ ]:
dataset_dict["train"].shape

(4348, 5)

In [ ]:
dataset_dict["train"][0]

{'index': '632.4',
 'user_id': '512486219',
 'text': "під час дихання та кашлю об'єктивно дихання",
 'audio': {'path': '512486219/632.4.wav',
  'array': array([ 0.00488281,  0.00112915, -0.00778198, ...,  0.006073  ,
          0.00598145,  0.00567627]),
  'sampling_rate': 44100},
 '__index_level_0__': 3149}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
dataset_dict.push_to_hub(
    hugginface_repo
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/7 [00:00<?, ?it/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Map:   0%|          | 0/621 [00:00<?, ? examples/s]

...

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Map:   0%|          | 0/543 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/546 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/lpetrl/UKR-Medical-Speech/commit/f68eeed0839d2397816eb7414817ebc79d075be3', commit_message='Upload dataset', commit_description='', oid='f68eeed0839d2397816eb7414817ebc79d075be3', pr_url=None, pr_revision=None, pr_num=None)